# Algorytm mrówkowy (ACO)

## Biblioteki

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import random
from openpyxl import load_workbook

## Pobór danych

In [2]:
file_path1 = "./Dane_TSP_48.xlsx"
file_path2 = "./Dane_TSP_76.xlsx"
file_path3 = "./Dane_TSP_127.xlsx"

# index_col=0 zamienia pierwszą kolumne na indeksy wierszy 
# .to_numpy() zamienia ramkę danych na macierz
data1 = pd.read_excel(file_path1, index_col=0).to_numpy()
data2 = pd.read_excel(file_path2, index_col=0).to_numpy()
data3 = pd.read_excel(file_path3, index_col=0).to_numpy()

## Długość ścieżki

In [3]:
def calculate_path_cost(matrix, path):
    return sum(matrix[path[i - 1]][path[i]] for i in range(len(path))) + matrix[path[-1]][path[0]]

## Implementacja algorytmu

In [4]:
def ant_colony_optimization(dist_matrix, num_ants=10, num_iterations=100, alpha=1, beta=2, rho=0.5):
    start_time = time.time()
    num_cities = len(dist_matrix)
    pheromone = np.ones((num_cities, num_cities)) / num_cities  # Inicjalizacja feromonów
    best_path = None
    best_path_length = float('inf')

    def select_next_city(visited, current_city):
        probabilities = []
        for next_city in range(num_cities):
            if next_city not in visited:
                tau = pheromone[current_city][next_city] ** alpha
                eta = (1 / dist_matrix[current_city][next_city]) ** beta
                # Dodanie prawdopodobieństwa wyboru miasta
                probabilities.append(tau * eta)
            else:
                # Jeśli miasto zostało odwiedzone, prawdopodobieństwo wynosi 0
                probabilities.append(0)
        total = sum(probabilities)
        probabilities = [p / total for p in probabilities]
        return random.choices(range(num_cities), probabilities)[0]

    for iteration in range(num_iterations):
        all_paths = [] # Lista ścieżek dla wszystkich mrówek w danej iteracji
        all_path_lengths = [] # Lista długości tych ścieżek
        
        for ant in range(num_ants):
            path = [] # Ścieżka odwiedzanych miast przez mrówkę
            visited = set()  # Zbiór odwiedzonych miast
            current_city = random.randint(0, num_cities - 1) # Losowy startowy wierzchołek
            path.append(current_city)
            visited.add(current_city)
            
            # Budowanie ścieżki - dopóki wszystkie miasta nie zostaną odwiedzone
            while len(visited) < num_cities:
                next_city = select_next_city(visited, current_city)
                path.append(next_city)
                visited.add(next_city)
                current_city = next_city

            # Obliczenie długości ścieżki mrówki
            path_length = calculate_path_cost(dist_matrix, path)
            all_paths.append(path)
            all_path_lengths.append(path_length)

             # Aktualizacja najlepszej ścieżki
            if path_length < best_path_length:
                best_path_length = path_length
                best_path = path

        # Aktualizacja feromonów
        pheromone *= (1 - rho)  # Parowanie feromonów (redukcja wartości)
        for path, length in zip(all_paths, all_path_lengths):
            for i in range(len(path)):
                pheromone[path[i]][path[(i + 1) % num_cities]] += 1 / length


    
    total_time = time.time() - start_time

    best_path = [city + 1 for city in best_path]


    return best_path, best_path_length, total_time

## Generowanie wyników

Poniżej znajduje się zestawienie wyników z uwzględnieniem wpływu na wyniki różnych wartości parametrów algorytmu

Za podstawowe dane przyjmujemy:

* num_ants = 10

* num_iterations = 100

* alpha = 1

* beta = 2

* rho = 0.5

Następnie badane są wpływy zmian wartości poszczególnych parametrów 

In [5]:
def run_multiple_tests(data, dataset_name, num_tests=10):
    results = []
    for _ in range(num_tests):
        best_path, best_cost, best_time = ant_colony_optimization(data)
        results.append({
            "WYNIK": best_cost,
            "ŚCIEŻKA": best_path,
            "CZAS": best_time,
            "DATASET": dataset_name
        })
    return results

In [6]:
all_results = []
all_results.extend(run_multiple_tests(data1, "DATA1"))
all_results.extend(run_multiple_tests(data2, "DATA2"))
all_results.extend(run_multiple_tests(data3, "DATA3"))

m1 = pd.DataFrame(all_results)

### Badanie wpływu zmian wartości parametru `num_ants`

In [ ]:
def test_ants_effect(data, dataset_name, num_repeats = 10):
    results = []
    num_cities = len(data)
    for num_ants in range(max(1, int(0.5 * num_cities)), int(2 * num_cities) + 1, 20):
        for _ in range(num_repeats):
            best_path, best_cost, best_time = ant_colony_optimization(data, num_ants=num_ants, num_iterations=100, alpha=1, beta=2, rho=0.5)
            results.append({"PARAMETR": num_ants, "WYNIK_1": best_cost, "ŚCIEŻKA": best_path, "CZAS": best_time})

    # Konwersja wyników na DataFrame
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Testy dla data1, data2, data3
df1 = test_ants_effect(data1, "DATA1")
df2 = test_ants_effect(data2, "DATA2")
df3 = test_ants_effect(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
ants = pd.concat([df1, df2, df3], ignore_index=True)

### Badanie wpływu zmian wartości parametru `num_iterations`

In [ ]:
def test_iteration(data, dataset_name, num_repeats = 10):
    results = []
   
    for num_iterations in range(50, 500, 50):
        for _ in range(num_repeats):
            best_path, best_cost, best_time = ant_colony_optimization(data, num_ants=10, num_iterations=num_iterations, alpha=1, beta=2, rho=0.5)
            results.append({"PARAMETR": num_iterations, "WYNIK_1": best_cost, "ŚCIEŻKA": best_path, "CZAS": best_time})

    # Konwersja wyników na DataFrame
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Testy dla data1, data2, data3
df1 = test_iteration(data1, "DATA1")
df2 = test_iteration(data2, "DATA2")
df3 = test_iteration(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
it = pd.concat([df1, df2, df3], ignore_index=True)

### Badanie wpływu zmian wartości parametru `alpha`

In [ ]:
def test_alpha(data, dataset_name, num_repeats= 10):
    results = []
   
    for alpha in range(0, 5, 1):
        for _ in range(num_repeats):
            best_path, best_cost, best_time = ant_colony_optimization(data, num_ants=10, num_iterations=100, alpha=alpha, beta=2, rho=0.5)
            results.append({"PARAMETR": alpha, "WYNIK_1": best_cost, "ŚCIEŻKA": best_path, "CZAS": best_time})

    # Konwersja wyników na DataFrame
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Testy dla data1, data2, data3
df1 = test_alpha(data1, "DATA1")
df2 = test_alpha(data2, "DATA2")
df3 = test_alpha(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
al = pd.concat([df1, df2, df3], ignore_index=True)

### Badanie wpływu zmian wartości parametru `beta`

In [ ]:
def test_beta(data, dataset_name, num_repeats=10):
    results = []
   
    for beta in range(1, 10, 1):
        for _ in range(num_repeats):
            best_path, best_cost, best_time = ant_colony_optimization(data, num_ants=10, num_iterations=100, alpha=1, beta=beta, rho=0.5)
            results.append({"PARAMETR": beta, "WYNIK_1": best_cost, "ŚCIEŻKA": best_path, "CZAS": best_time})

    # Konwersja wyników na DataFrame
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Testy dla data1, data2, data3
df1 = test_beta(data1, "DATA1")
df2 = test_beta(data2, "DATA2")
df3 = test_beta(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
bet = pd.concat([df1, df2, df3], ignore_index=True)

### Badanie wpływu zmian wartości parametru `rho`

In [ ]:
def test_rho(data, dataset_name, num_repeats=10):
    results = []
   
    for rho in np.linspace(0.1, 0.9, 9):
        for _ in range(num_repeats):
            best_path, best_cost, best_time = ant_colony_optimization(data, num_ants=10, num_iterations=100, alpha=1, beta=2, rho=rho)
            results.append({"PARAMETR": rho, "WYNIK_1": best_cost, "ŚCIEŻKA": best_path, "CZAS": best_time})

    # Konwersja wyników na DataFrame
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Testy dla data1, data2, data3
df1 =test_rho(data1, "DATA1")
df2 =test_rho(data2, "DATA2")
df3 = test_rho(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
rho = pd.concat([df1, df2, df3], ignore_index=True)

Zapis do pliku Excel

In [ ]:
result = {
    "Klasyk_wyniki": m1,
    "num_ants": ants,
    "max_it":  it,
    "alpha": al,
    "beta": bet,
    "rho": rho
}

# Ścieżka do pliku Excel
file_name = "ACO.xlsx"

# Zapisujemy dane do Excela
with pd.ExcelWriter(file_name) as writer:
    for sheet_name, df in result.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Wyniki zostały zapisane w pliku {file_name}")